18대 19대 의회 회의록 전처리 코드

1. Colab과 google drive 연결; 구글 드라이브에서 공유하고 있는 회의록 파일을 Colab에 연동하여 분석하기 위한 사전 작업.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

2. 각 회의록 파일 불러오기: 내 드라이브에서 각 파일을 불러와 Files라는 list에 저장하는 코드



In [ ]:
 from os import listdir
 from os.path import isfile, join
 filespath = '/content/drive/My Drive/all'
 files = [f for f in listdir(filespath) if isfile(join(filespath, f))]

 filepath=[]
 for file in files:
     filepath.append(filespath + '/' + file)

 Files = []
 for i in filepath:
     f = open(i, encoding='CP949')
     text = f.read()
     Files.append(text)
     f.close

3. 전처리 코드 함수화: 각 회의록에 대해서 다음과 같은 작업을 수행함
- 발언 기호 ◯("\n.")를 기준으로 텍스트를 나눔
- 나눈 텍스트에서 발언자와 발언을 분리함
- 띄어쓰기를 기준으로 길이를 잼
- 특정한 단어의 횟수를 기록함(단어를 띄어쓰기 단위로 나누었다)


In [ ]:
len(Files)

3160

In [ ]:
def cleaner(text, j):
    import re
    import pandas as pd
    text=str(text)
    text_split=text.split('\n.')

    name = []
    talk = []
    length = []
    
    for i in range(0, len(text_split)):
        name.append(" ".join(text_split[i].split()[:2]))
        talk.append(" ".join(text_split[i].split()[2:]))
        length.append(len(text_split[i].split()[2:]))
    
    data = {
        'NAME': name,
        'TALK': talk,
        'LENGTH': length,
        'FILENAME' : filepath[j],
        'INDEX_IN_MEETING' : list(range(0, len(text_split))),
        'WHEN' : talk[0].split(' ')[1],
        'WHERE' : talk[0].split(' ')[2]
        }
    d=pd.DataFrame(data)
    return d

4. 한 회의록 파일 확인, 함수에 필요한 키워드 변수 할당


In [ ]:
start = cleaner(Files[0], 0)

In [ ]:
import pandas as pd

for j in range(len(Files)-2):   
    start = pd.concat([start, cleaner(Files[j+1], j+1)])

In [ ]:
start.shape

(1477086, 7)

In [ ]:
all = start

In [ ]:
# all.to_csv("18-19congress_cleaned_0924.csv")

In [ ]:
# all.to_csv("18-19congress_cleaned_0924_utf8.csv", encoding="utf-8")

give whole index to aggregated dataframe

In [ ]:
all['INDEX'] = list(range(len(all)))

make round variables and committe variable out of filepaths


In [ ]:
all['filepath_resi_front'] = start.FILENAME.str.split('_').str[0]
all['ROUND1'] = start.FILENAME.str.split('_').str[1]
all['ROUND2'] = start.FILENAME.str.split('_').str[2]
all['ROUND3'] = start.FILENAME.str.split('_').str[3]
all['COMMITTEE'] = start.FILENAME.str.split('_').str[4]
all['filepath_resi_back'] = start.FILENAME.str.split('_').str[5]

In [ ]:
all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1477086 entries, 0 to 635
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   NAME                 1477086 non-null  object 
 1   TALK                 1477086 non-null  object 
 2   LENGTH               1477086 non-null  int64  
 3   FILENAME             1477086 non-null  object 
 4   INDEX_IN_MEETING     1477086 non-null  int64  
 5   WHEN                 1477086 non-null  object 
 6   WHERE                1477086 non-null  object 
 7   INDEX                1477086 non-null  int64  
 8   filepath_resi_front  1477086 non-null  object 
 9   ROUND1               1477086 non-null  object 
 10  ROUND2               1477086 non-null  object 
 11  ROUND3               1477086 non-null  object 
 12  COMMITTEE            1477086 non-null  object 
 13  filepath_resi_back   0 non-null        float64
dtypes: float64(1), int64(3), object(10)
memory usage: 169.

In [ ]:
all.to_csv("18-19congress_time_0924.csv")

In [ ]:
all.to_csv("18-19congress_time_utf8_0924.csv", encoding="utf-8")

In [ ]:
#-----0924-----